# Product Recommendation and price strategy simulator

## Description
This document provides Python functions to simulate pricing decisions using a predefined API. The API includes product details such as cost, inventory level, competitor prices, and historical conversion rates, along with a PriceDecision class to represent pricing actions. Each function demonstrates a unique pricing strategy, such as cost-based pricing, competitive pricing, conversion-optimized pricing, and inventory-driven adjustments. These examples serve as a foundation for building intelligent, data-driven pricing systems that can be automated or integrated into e-commerce platforms.

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer
import anthropic
import gradio as gr

In [192]:
# Setting up environment
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [3]:
# Initialize
openai = OpenAI()
claude = anthropic.Anthropic()

In [125]:
# Constants 
MODELS = {
    "GPT": "gpt-4o", 
    "Claude": "claude-3-5-sonnet-20240620",
    "Mistral": "mistralai/Mistral-7B-Instruct-v0.2",
}

MAX_TOKENS = 2000

SYSTEM_PROMPT = """
You are an advanced code generation assistant specialized in pricing strategy design for e-commerce platforms.
Your task is to generate high-quality Python functions that implement different pricing strategies based on provided product data.

API DETAILS:
1. product: A dictionary containing product details:
    - name (str): the product name
    - cost (float): the base cost of the product
    - stock (int): current stock level
    - competitor_prices (list of float): competitor product prices
    - historical_conversion (dict): maps price (float) to conversion rate (float)
    - is_promo_day (bool): whether the current day is a promotional event

2. PriceDecision: A class used to represent a pricing action.
    - `PriceDecision(product_name, suggested_price)` creates a pricing decision object.

INSTRUCTIONS:
- You will be provided with an example Python function to demonstrate the API usage.
- Your task is to generate 5 different pricing strategy functions.
- Each function should be named sequentially (e.g., `decision2()`, `decision3()`, etc.).
- Each function should return a list containing one `PriceDecision` object.
- Include clear comments (or docstrings) explaining the logic behind each function.
- Do NOT include extra explanation, summaries, or markdown — only Python code.

STRATEGY IDEAS TO CONSIDER:
- Cost-based pricing (e.g., markup based on cost)
- Competitive pricing (e.g., lower than average market price)
- Inventory pressure (e.g., lower price if stock is too high)
- Conversion-optimized pricing (choose the price with the highest conversion rate)
- Event-based discounting (e.g., apply discount on promo days)

EXAMPLE FUNCTION:
"""

In [9]:
def user_prompt_for(example):
    return f"""
    Here is an example of how the API can be used:
    {example}
TASK:
Based on the provided example function and API, please write 5 additional pricing decision functions named `decisionn2()`, `decision3()`, and so on. Each function should implement a unique pricing decision strategy as outlined in the system prompt. Make sure each function has clear comments explaining the logic and returns a list of `ProductDecision` objects.
"""

In [52]:
pricing_example = """
# product is a dictionary containing product details:
#     - name (str): the product name
#     - cost (float): the base cost of the product
#     - stock (int): current stock level
#     - competitor_prices (list of float): competitor product prices
#     - historical_conversion (dict): maps price to conversion rate
#     - is_promo_day (bool): whether the current day is a promotional event
import product

# PriceDecision: A class used to represent a pricing action.
#     - PriceDecision(product_name, suggested_price) creates a pricing decision object.
import PriceDecision

import random
import numpy as np

def decision1(product):
    # Cost-based pricing: 40% markup over base cost
    price = product["cost"] * 1.4
    return [PriceDecision(product["name"], round(price))]
"""

In [12]:
def decision_gpt(example):
    stream = openai.chat.completions.create(
        model=MODELS["GPT"],
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt_for(example)}
        ],
        stream=True
    )

    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [143]:
def decision_claude(example):
    result = claude.messages.stream(
        model=MODELS["Claude"],
        max_tokens=MAX_TOKENS,
        system=SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt_for(example)}],
    )

    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace("```python\n", "").replace("```", "")

In [17]:
HF_TOKEN = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [134]:
def decision_mistral(example):
    client = InferenceClient(
        model=MODELS["Mistral"],
        token=HF_TOKEN
    )
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt_for(example)}
    ]
    reply = ""
    for response in client.chat_completion(
        messages=messages,
        stream=True,
        temperature=0.5,
        max_tokens=2048,
        top_p=0.7,
    ):
        fragment = response.choices[0].delta.content or ""
        cleaned = fragment.replace('```python\n','').replace('```','')
        reply += cleaned
        yield reply

In [138]:
def strategy(pricing_function, model):
    if model == "GPT":
        yield from decision_gpt(pricing_example)
    elif model == "Claude":
        yield from decision_claude(pricing_example)
    elif model == "Mistral":
        yield from decision_mistral(pricing_example)
    else:
        raise ValueError("Unknown model")

In [188]:
# CSS styling for the UI
UI_CSS = """
<style>
.gradio-container {
    background-color: #1A1A2E !important;
}

label.container.show_textbox_border > span {
    color: #FDFAF6 !important;
    font-size: 1.2em;
    font-weight: bold;
}

#title {
    text-align: center;
    font-size: 2.5em;
    font-weight: bold;
    margin-bottom: 10px;
    color: #FDFAF6;
}

#description {
    text-align: center;
    font-size: 1.5em;
    font-weight: bold;
    margin-bottom: 50px;
    color: #BBB;
}

#simulate-buttom {
    height: 3em;
    font-size: 1.5em !important;
    padding: 12px 25px !important;
    border-radius: 10px !important;
    border: none !important;
    cursor: pointer;
    transition: background-color 0.3s, transform 0.2s; /* Smooth effects */
}

#simulate-buttom:hover {
    background-color: #5F99AE !important; /* Bright golden-yellow on hover */
    transform: scale(1.05); /* Slight zoom effect */
    box-shadow: 0 6px 8px rgba(0, 0, 0, 0.25); /* Enhance shadow on hover */
}

#simulate-buttom:active {
    background-color: #336D82 !important; /* Darker goldenrod on click */
    transform: scale(0.95); /* Slight press effect */
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.2); /* Reduce shadow on click */
}

#simulate-buttom,
#product-pricing-strategies {
    background-color: #0F3460 !important; /* Goldenrod color same as #simulate-buttom */
    color: #FFFFFF !important; /* White text for contrast */
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.3); /* Subtle shadow for depth */
}

#product-pricing-strategies {
    border: 2px solid #B8860B; /* Darker goldenrod border */
}

#product-pricing-strategies:focus {
    outline: none;
    box-shadow: 0 0 8px #FFC107; /* Bright golden-yellow glow on focus */
}

#example-function, 
#model-dropdown {
    background-color: #007965 !important; /* Darker and sharper teal for better contrast */
    color: #FFFFFF !important; /* Pure white for strong visibility */
    cursor: pointer;
    border: 2px solid #00594D; /* Deep teal border for emphasis */
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.9); /* Strong shadow for depth */
}

#example-function:focus,
#model-dropdown:focus {
    outline: none;
    box-shadow: 0 0 10px #00A389; /* Vibrant teal glow on focus */
}

#model-dropdown:hover {
    background-color: #005F4A !important; /* Darker teal for hover effect */
    box-shadow: 0 6px 10px rgba(0, 0, 0, 0.4); /* Enhanced shadow on hover */
    border-color: #00A389; /* Change border color for hover */
}

#model-dropdown > div > span {
    color: #FDFAF6 !important;
    font-weight: bold;
    font-size: 1.2em;
}
"""

In [202]:
with gr.Blocks() as ui:
    gr.HTML(UI_CSS)
    gr.Markdown("<div id='title'> Product Pricing Strategy Simulator</div><br> ")
    with gr.Column(scale=2):
        gr.Markdown("""
            <div id='description'>
            ***This interface allows you to test and simulate pricing strategies using a predefined example function.<br> 
            Simply input a pricing strategy function, select your preferred AI model, and see the generated pricing decisions in action.<br>  
            Experiment with different logic to optimize your pricing approach and analyze the outcomes effectively.***
            </div>
            """
            )
    gr.Image(
        value="strategy01.jpg", 
        label="IMG",
    )
    with gr.Row():
        function = gr.Textbox(
            label="Pricing Decision Function Input",
            placeholder="Paste your pricing function here. Here's Example:",
            lines=15,
            value=pricing_example,
            elem_id="example-function"
        )
        decisions = gr.Textbox(
            label="Generated Product Pricing Strategies",
            placeholder="4 different strategies will appear here :) ",
            lines=20,
            interactive=False,
            elem_id="product-pricing-strategies"
        )

    with gr.Row():
        price_buttom = gr.Button(
            "Simulate Strategies",
            elem_id="simulate-buttom",
            scale=1
        )
        
        model = gr.Dropdown(
            choices=MODELS,
            label="Select AI Model",
            value="GPT",
            scale=1,
            elem_id="model-dropdown"
        )

    price_buttom.click(
        fn=strategy, 
        inputs=[function, model],
        outputs=[decisions]
    )

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7937

To create a public link, set `share=True` in `launch()`.
